In [1]:
#%%
import os
import sys
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import openpyxl
import pandas as pd
import math
import itertools
import matplotlib.patches as patches
import seaborn as sns
from decimal import Decimal
from csv import reader
import csv
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.distribution import MultiprocessingDistributor
from scipy.interpolate import interp1d



time_units =0.01

def get_app_list(fileName):
    """get metrics list

    Arguments:
        fileName {string} -- the file name of application file with absolute path

    Returns:
        app {list} -- the app  list
    """
    apps = []
    with open(fileName) as f:
        for line in f.readlines():
            if not line.startswith('#'):
                words = line.strip().split(',')
                app = words[0].strip()
                app_num = words[1].strip()
                apps.append(app+app_num)
        f.close()
    return apps

def trace_read_C(fileName, power_outName):
    # index
  
    data = []
    repeat_data = []
    with open(fileName) as f:      
        allLines = f.readlines()
        n = len(allLines)        
        if n <=2:
            print(fileName)
            f.close()
            return
        
        
        t_all = float(allLines[n-1])
        t = 0
        t_step = t_all/(n-2)
        
        for i in range(1, n-1):
            line = allLines[i]
            temp = line.split(',')
            temp_row = [float(i.split()[0]) for i in temp]
            temp_row.insert(0, t)
            data.append(temp_row)
            t+=t_step
        f.close()
        
        i=0
        while t < 360:
            t += t_step
            data.append([t, data[i][1], data[i][2], data[i][3], data[i][4]] )
            i += 1
            i = i % (n-2)
    
    columns= ["time", "power", "memory Used", "u_gpu", "u_memory"]
    df = pd.DataFrame(data=data, columns=columns)
    df.to_csv(power_outName,index=False)
    

def trace_read(app_list, pathfolder):
    for app in app_list:
        fileName = os.path.join(pathfolder, "{}.pwr.txt".format(app))
        if not os.path.exists(fileName):
            print(fileName)
            continue
        power_outName = os.path.join(pathfolder, "{}_res.csv".format(app))      
        trace_read_C(fileName, power_outName)

 
    #df.to_csv(outFile, index=False )

def load_file(filepath):
    
    dataframe = pd.read_csv(filepath, index_col= None)
    if (dataframe.shape[1] == 5 ):
        #if dataframe["u_gpu"].sum() == 0:
        #    #os.remove(filepath)
        #    print("1", filepath)
        #else:
            return dataframe.values[:,:]
    else:
        print("2", filepath)
def interpolate(app_list, pathfolder, sample_rate):
    max_len = 359*1000//sample_rate
    max_len = 1800
    
    x_new = np.linspace(0, 359, max_len)
    columns= ["power", "memory Used", "u_gpu", "u_memory"]
    for app in app_list:
        #print(app)
        power_outName = os.path.join(pathfolder, "{}_res.csv".format(app))     
        inter_outName = os.path.join(pathfolder, "{}_int.csv".format(app))     
        data = load_file(power_outName)        
        x_old = data[:, 0]
        data_new = []
        for i in range(1, 5):
            #print(data[:, i])
            ius = interp1d(x_old, data[:, i], kind='nearest')
            #ius = InterpolatedUnivariateSpline(x_old, data[:, i])
            y = ius(x_new)
            data_new.append(y)
            #print(y)

        #print(data_new)
        df = pd.DataFrame(data_new).T
        
        df.columns = columns
        df.to_csv(inter_outName,index=False)
        
if __name__=="__main__":        
    category = "risky"
    for sample_rate in [100]: #, 50, 10
        for category in ["mybench", "risky"]:#, "risky"]:
            for arch in ["k40", "v100", "p100"]:#, ]:
        #arch ="v100"/home/pzou/projects/Power_Signature/results/Turing/mem_trace-combine
                pathfolder = r"/home/pzou/projects/Power_Signature/results-%d/%s/%s/power-combine"%(sample_rate, category, arch)
                app_list = get_app_list("/home/pzou/projects/Power_Signature/Scripts/applications_%s.csv"%(category))
                #trace_read(app_list, pathfolder)
                interpolate(app_list, pathfolder, sample_rate)
                print("end")
            
#%%


/software/anaconda3/5.1.0/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


end
end
end
end
end
end
